# How to load data to BigQuery

Here we demonstrate a few different ways to load data to BigQuery from an R notebook.
* [bq](https://cloud.google.com/bigquery/docs/bq-command-line-tool) command line tool
* [bigrquery](https://cloud.google.com/blog/products/gcp/google-cloud-platform-for-data-scientists-using-r-with-google-bigquery-part-2-storing-and-retrieving-data-frames)

## Setup

First, be sure to run notebook **`R environment setup`** in this workspace.

Then in this section we:

1. load the needed R packages
2. set the project id of the cloud project to bill for queries to BigQuery
3. authorize our bigquery client library to issue requests

In [1]:
library(jsonlite)
library(bigrquery)
library(lubridate)
library(tidyverse)


Attaching package: ‘lubridate’


The following object is masked from ‘package:base’:

    date


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ purrr::flatten()         masks jsonlite::flatten()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()



Edit these global variables in your clone of this notebook if you do not have permission to WRITE data to this native Google Cloud Platform project.
* The destination BigQuery dataset should already exist. Your pet account must have WRITE access to it.
* The remaining cells can be run as-is.

In [2]:
# CHANGE THESE VARIABLES, IF NEEDED
DESTINATION_PROJECT_ID <- 'terra-resources'
DESTINATION_DATASET <- 'autodelete_after_one_day'

In [3]:
# This file loads fine via autodetect.
CSV_PATH <- 'gs://genomics-public-data/platinum-genomes/other/platinum_genomes_sample_info.csv'

# Also try this CSV which will yield some autodetect errors.
CSV_PATH_AUTODETECT_FAILS <- 'gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv'

BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

# Load data to BigQuery from a CSV

## Via the `bq` command line tool and autodetect

In [4]:
DESTINATION_TABLE <- paste0('r_bq_autodetect_', strftime(now(), '%Y%m%d_%H%M%S'))

In [5]:
system(str_glue(str_c('bq --project {BILLING_PROJECT_ID} load ',
                      '--autodetect ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE} ',
                      '{CSV_PATH}  2>&1')),
      intern = TRUE)

[1] "\rWaiting on bqjob_r1b8ea0a5168caab8_0000016fa0b0d7cf_1 ... (0s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r1b8ea0a5168caab8_0000016fa0b0d7cf_1 ... (1s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r1b8ea0a5168caab8_0000016fa0b0d7cf_1 ... (1s) Current status: DONE   "

Show the table schema.

In [6]:
system(str_glue(str_c('bq --project {BILLING_PROJECT_ID} show ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE}')),
      intern = TRUE)

[1] "Table terra-resources:autodelete_after_one_day.r_bq_autodetect_20200113_205535"                                                                       
 [2] ""                                                                                                                                                     
 [3] "   Last modified                  Schema                 Total Rows   Total Bytes     Expiration      Time Partitioning   Clustered Fields   Labels  "
 [4] " ----------------- ------------------------------------ ------------ ------------- ----------------- ------------------- ------------------ -------- "
 [5] "  13 Jan 20:55:44   |- Catalog_ID: string                17           1441          14 Jan 20:55:44                                                  "
 [6] "                    |- Description: string                                                                                                           "
 [7] "                    |- Gender: string                                                                                                                "
 [8] "                    |- Race: string                                                                                                                  "
 [9] "                    |- Family_Member: integer                                                                                                        "
[10] "                    |- Relationship_to_Proband: string                                                                                               "
[11] "                    |- CT_Desc: string                                                                                                               "
[12] "                    |- Cum_Pdl: string                                                                                                               "
[13] ""

## Via the `bq` command line tool with a manual schema

Right now there is currently not a way to tell `bq --autodetect` to examine more rows. Instead, use R to autodetect the schema and then pass that detected schema to `bq`

In [7]:
DESTINATION_TABLE <- paste0('r_bq_manual_schema_', strftime(now(), '%Y%m%d_%H%M%S'))

In [8]:
system(str_glue(str_c('bq --project {BILLING_PROJECT_ID} load ',
                      '--autodetect ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE} ',
                      '{CSV_PATH_AUTODETECT_FAILS} 2>&1')),
      intern = TRUE)

Warning message in system(str_glue(str_c("bq --project {BILLING_PROJECT_ID} load ", :
“running command 'bq --project fc-product-demo load --autodetect terra-resources:autodelete_after_one_day.r_bq_manual_schema_20200113_205546 gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv 2>&1' had status 1”


[1] "\rWaiting on bqjob_r70d15955e8565724_0000016fa0b0eb3a_1 ... (0s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r70d15955e8565724_0000016fa0b0eb3a_1 ... (1s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r70d15955e8565724_0000016fa0b0eb3a_1 ... (2s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r70d15955e8565724_0000016fa0b0eb3a_1 ... (2s) Current status: DONE   "
 [2] "BigQuery error in load operation: Error processing job"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "'fc-product-demo:bqjob_r70d15955e8565724_0000016fa0b0eb3a_1': Error while"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 [4] "reading data, error message: CSV table encountered too many errors, giving up."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "Rows: 806; errors: 1. Please look into the errors[] collection for more details."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [6] "Failure details:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [7] "- gs://genomics-public-"                                                                                                                                                                                                                                   

And we see that `--autodetect` fails for this CSV.

In [9]:
NUM_ROWS <- 3000
# TODO if the file is very large, grab a subset of it via gsutil cat {CSV_PATH_AUTODETECT_FAILS} | head {NUM_ROWS}
df <- read_csv(pipe(str_glue('gsutil cat {CSV_PATH_AUTODETECT_FAILS}')),
               guess_max = NUM_ROWS)

Parsed with column specification:
cols(
  .default = col_character(),
  In_Low_Coverage_Pilot = col_double(),
  In_High_Coverage_Pilot = col_double(),
  In_Exon_Targetted_Pilot = col_double(),
  Has_Sequence_in_Phase1 = col_double(),
  In_Phase1_Integrated_Variant_Set = col_double(),
  Has_Phase1_chrY_SNPS = col_double(),
  Has_phase1_chrY_Deletions = col_double(),
  Has_phase1_chrMT_SNPs = col_double(),
  Total_LC_Sequence = col_double(),
  LC_Non_Duplicated_Aligned_Coverage = col_double(),
  Total_Exome_Sequence = col_double(),
  X_Targets_Covered_to_20x_or_greater = col_double(),
  VerifyBam_E_Omni_Free = col_double(),
  VerifyBam_E_Affy_Free = col_double(),
  VerifyBam_E_Omni_Chip = col_double(),
  VerifyBam_E_Affy_Chip = col_double(),
  VerifyBam_LC_Omni_Free = col_double(),
  VerifyBam_LC_Affy_Free = col_double(),
  VerifyBam_LC_Omni_Chip = col_double(),
  VerifyBam_LC_Affy_Chip = col_double()
  # ... with 11 more columns
)

See spec(...) for full column specifications.



In [10]:
bq_schema <- tibble(name=colnames(df),
                    raw_type=unlist(map(df, typeof))) %>%
    mutate(
        mode = 'NULLABLE',
        type = case_when(
            raw_type == 'character' ~ 'STRING',
            raw_type == 'integer' ~ 'INT64',
            raw_type == 'double' ~ 'FLOAT64',
            raw_type == 'logical' ~ 'BOOLEAN',
            TRUE ~ raw_type
        )
    )

head(bq_schema)

name,raw_type,mode,type
<chr>,<chr>,<chr>,<chr>
Sample,character,NULLABLE,STRING
Family_ID,character,NULLABLE,STRING
Population,character,NULLABLE,STRING
Population_Description,character,NULLABLE,STRING
Gender,character,NULLABLE,STRING
Relationship,character,NULLABLE,STRING


In [11]:
table(bq_schema$raw_type)


character    double 
       31        31 

In [12]:
table(bq_schema$type)


FLOAT64  STRING 
     31      31 

In [13]:
schema_file <- tempfile(fileext = '.json')
schema_file

[1] "/tmp/RtmpG5vzkT/file15023f1d18e.json"

In [14]:
write(x = toJSON(bq_schema %>% select(-raw_type)), file = schema_file)

In [15]:
system(str_glue(str_c(
    'bq --project {BILLING_PROJECT_ID} load ',
    '--schema {schema_file} ',
    '--skip_leading_rows 1 ',
    '--source_format CSV ',
    '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE} ',
    '{CSV_PATH_AUTODETECT_FAILS} 2>&1'
)), intern = TRUE)

[1] "\rWaiting on bqjob_rbc550d1ab01a065_0000016fa0b12601_1 ... (0s) Current status: RUNNING\r                                                                                     \rWaiting on bqjob_rbc550d1ab01a065_0000016fa0b12601_1 ... (1s) Current status: RUNNING\r                                                                                     \rWaiting on bqjob_rbc550d1ab01a065_0000016fa0b12601_1 ... (1s) Current status: DONE   "

Show the table schema.

In [16]:
system(str_glue(str_c('bq --project {BILLING_PROJECT_ID} show ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE}')),
      intern = TRUE)

[1] "Table terra-resources:autodelete_after_one_day.r_bq_manual_schema_20200113_205546"                                                                               
 [2] ""                                                                                                                                                                
 [3] "   Last modified                       Schema                       Total Rows   Total Bytes     Expiration      Time Partitioning   Clustered Fields   Labels  "
 [4] " ----------------- ----------------------------------------------- ------------ ------------- ----------------- ------------------- ------------------ -------- "
 [5] "  13 Jan 20:56:04   |- Sample: string                               3500         965676        14 Jan 20:56:04                                                  "
 [6] "                    |- Family_ID: string                                                                                                                        "
 [7] "                    |- Population: string                                                                                                                       "
 [8] "                    |- Population_Description: string                                                                                                           "
 [9] "                    |- Gender: string                                                                                                                           "
[10] "                    |- Relationship: string                                                                                                                     "
[11] "                    |- Unexpected_Parent_Child: string                                                                                                          "
[12] "                    |- Non_Paternity: string                                                                                                                    "
[13] "                    |- Siblings: string                                                                                                                         "
[14] "                    |- Grandparents: string                                                                                                                     "
[15] "                    |- Avuncular: string                                                                                                                        "
[16] "                    |- Half_Siblings: string                                                                                                                    "
[17] "                    |- Unknown_Second_Order: string                                                                                                             "
[18] "                    |- Third_Order: string                                                                                                                      "
[19] "                    |- In_Low_Coverage_Pilot: float                                                                                                             "
[20] "                    |- LC_Pilot_Platforms: string                                                                                                               "
[21] "                    |- LC_Pilot_Centers: string                                                                                                                 "
[22] "                    |- In_High_Coverage_Pilot: float                                                                                                            "
[23] "                    |- HC_Pilot_Platforms: string                                                                                                               "
[24] "                    |- HC_Pilot_Centers: string                                                                                    

# Load data to BigQuery from a dataframe

## Via bigrquery

In [17]:
DESTINATION_TABLE <- paste0('r_bigrquery_', strftime(now(), '%Y%m%d_%H%M%S'))

In [18]:
insert_upload_job(project = DESTINATION_PROJECT_ID,
                  # Researchers currently only have permission to write to a dataset whose tables are
                  # automatically cleaned up after one week.
                  dataset = DESTINATION_DATASET,
                  # Add your username to the table name to make the name unique to you.
                  table = DESTINATION_TABLE,
                  billing = BILLING_PROJECT_ID,
                  write_disposition = 'WRITE_EMPTY',
                  mtcars)

$kind
[1] "bigquery#job"

$etag
[1] "KFzXGn5a6OgkgfEkZ1Hc1g=="

$id
[1] "fc-product-demo:US.job_JNSn4EneOuJPe8dgrVtENiblpM3N"

$selfLink
[1] "https://www.googleapis.com/bigquery/v2/projects/fc-product-demo/jobs/job_JNSn4EneOuJPe8dgrVtENiblpM3N?location=US"

$user_email
[1] "pet-110502112294304402890@fc-product-demo.iam.gserviceaccount.com"

$configuration
$configuration$load
$configuration$load$schema
$configuration$load$schema$fields
$configuration$load$schema$fields[[1]]
$configuration$load$schema$fields[[1]]$name
[1] "mpg"

$configuration$load$schema$fields[[1]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[1]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[1]]$description
[1] ""


$configuration$load$schema$fields[[2]]
$configuration$load$schema$fields[[2]]$name
[1] "cyl"

$configuration$load$schema$fields[[2]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[2]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[2]]$description
[1] ""


$configuration$load$schema$fields[[3]]
$configuration$load$schema$fields[[3]]$name
[1] "disp"

$configuration$load$schema$fields[[3]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[3]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[3]]$description
[1] ""


$configuration$load$schema$fields[[4]]
$configuration$load$schema$fields[[4]]$name
[1] "hp"

$configuration$load$schema$fields[[4]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[4]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[4]]$description
[1] ""


$configuration$load$schema$fields[[5]]
$configuration$load$schema$fields[[5]]$name
[1] "drat"

$configuration$load$schema$fields[[5]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[5]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[5]]$description
[1] ""


$configuration$load$schema$fields[[6]]
$configuration$load$schema$fields[[6]]$name
[1] "wt"

$configuration$load$schema$fields[[6]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[6]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[6]]$description
[1] ""


$configuration$load$schema$fields[[7]]
$configuration$load$schema$fields[[7]]$name
[1] "qsec"

$configuration$load$schema$fields[[7]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[7]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[7]]$description
[1] ""


$configuration$load$schema$fields[[8]]
$configuration$load$schema$fields[[8]]$name
[1] "vs"

$configuration$load$schema$fields[[8]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[8]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[8]]$description
[1] ""


$configuration$load$schema$fields[[9]]
$configuration$load$schema$fields[[9]]$name
[1] "am"

$configuration$load$schema$fields[[9]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[9]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[9]]$description
[1] ""


$configuration$load$schema$fields[[10]]
$configuration$load$schema$fields[[10]]$name
[1] "gear"

$configuration$load$schema$fields[[10]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[10]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[10]]$description
[1] ""


$configuration$load$schema$fields[[11]]
$configuration$load$schema$fields[[11]]$name
[1] "carb"

$configuration$load$schema$fields[[11]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[11]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[11]]$description
[1] ""




$configuration$load$destinationTable
$configuration$load$destinationTable$projectId
[1] "terra-resources"

$configuration$load$destinationTable$datasetId
[1] "autodelete_after_one_day"

$configuration$load$destinationTable$tableId
[1] "r_bigrquery_20200113_205608"


$configuration$load$createDisposition
[1] "CREATE_IF_NEEDED"

$configuration$load$writeDisposition
[1] "WRITE_EMPTY"

$configuration$load$sourceFormat
[1] "NEWLINE_DELIMITED_JSON"


$configuration$jobType
[1] "LOAD"


$jobReference
$jobReference$projectId
[1] "fc-product-demo"

$jobReference$jobId
[

In [20]:
# Create a "connection" to a public BigQuery dataset.
dbcon <- bigrquery::src_bigquery(project = DESTINATION_PROJECT_ID,
                                 dataset = DESTINATION_DATASET,
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table.
tbl <- dplyr::tbl(dbcon, DESTINATION_TABLE)
colnames(tbl)

[1] "mpg"  "cyl"  "disp" "hp"   "drat" "wt"   "qsec" "vs"   "am"   "gear"
[11] "carb"

# Provenance

In [21]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 3.5.2 (2018-12-20)
 os       Debian GNU/Linux 9 (stretch)
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2020-01-13                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version    date       lib source                            
 assertthat    0.2.1      2019-03-21 [2] CRAN (R 3.5.2)                    
 backports     1.1.4      2019-04-10 [2] CRAN (R 3.5.2)                    
 base64enc     0.1-3      2015-07-28 [2] CRAN (R 3.5.2)                    
 bigrquery   * 1.2.0      2019-07-02 [2] CRAN (R 3.5.2)                    
 bit           1.1-14     2018-05-29 [2] CRAN (R 3.5.2)                

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.